## This notebook creates a table with monthly seascape class and cruise metadata, and renders data visualizations and seascape maps.

In [ ]:
# Load libraries
import pandas as pd
import numpy as np
import xarray as xr
from pathlib import Path
import matplotlib.pyplot as plt
import requests
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.colors import BoundaryNorm
from scipy.spatial import KDTree
from scipy.stats import mode as sp_mode

In [ ]:
# Load SFER data
# Define the path to the data file
ctd_file = Path('~/plankton_imaging/ctd_meta_v4.csv').expanduser()

# Load the data into a pandas DataFrame
ctd_df = pd.read_csv(ctd_file)

# Display the first few rows of the DataFrame to verify it loaded correctly
ctd_df.head()

# Print few rows of the DataFrame to verify it loaded correctly
print(ctd_df.head())

In [ ]:
# Change this variable to 'SEASCAPE_MONTH' or 'SEASCAPE_8DAY' as needed
dataset_type = 'SEASCAPE_8DAY' 

# Define Domain Limits and Dates 
n_lat = 29
s_lat = 23
e_lon = -79.5
w_lon = -86
start_date = '2022-11-15T12:00:00Z'
end_date = '2025-10-15T12:00:00Z'

# Build the Request for the MONTHLY Product 
print(f"Configuring request for: {dataset_type}")

base_url = f"https://cwcgom.aoml.noaa.gov/thredds/ncss/{dataset_type}/SEASCAPES.nc"

params = {
    'var': ['CLASS', 'P'],
    'north': n_lat,
    'west': w_lon,
    'east': e_lon,
    'south': s_lat,
    'disableProjSubset': 'on',
    'horizStride': 1,
    'time_start': start_date,
    'time_end': end_date,
    'timeStride': 1,
    'addLatLon': 'true',
    'accept': 'netcdf',
}

req = requests.Request('GET', base_url, params=params).prepare()
url = req.url

print("Requesting data from URL:")
print(url)

# Download the Data Manually and Then Open ---
output_filename = 'seascape_data.nc'

try:
    # Download the file using requests
    print(f"\nDownloading data to '{output_filename}'...")
    with requests.get(url, stream=True) as r:
        r.raise_for_status()  # This will raise an error for bad status codes
        with open(output_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    print("Download complete.")

    # Open the local file with xarray
    ds = xr.open_dataset(output_filename)
    print("\nSuccessfully loaded dataset from local file:")
    print(ds)

    # Analyze Unique Classes ---
    class_data = ds['CLASS'].values
    unique_classes_with_nan = np.unique(class_data)
    unique_classes = unique_classes_with_nan[~np.isnan(unique_classes_with_nan)]

    print("\nUnique classes present in CLASS:")
    print(unique_classes.astype(int))

except Exception as err:
    print(f"\nAn error occurred: {err}")

In [ ]:
# Create TIMEVEC DataFrame.
# Access the time coordinate from the xarray dataset
time_coordinate = ds['time']

# Create a dictionary with the year, month, day, and full datetime
time_data = {
    'year': time_coordinate.dt.year.values,
    'month': time_coordinate.dt.month.values,
    'day': time_coordinate.dt.day.values,
    'datetime': time_coordinate.values
}

# Convert the dictionary into a pandas DataFrame
TIMEVEC = pd.DataFrame(time_data)

print("Created TIMEVEC DataFrame:")
print(TIMEVEC)

In [ ]:
#  Map Seascape Data for a Single Time Step as a test

# Select the Data for a Single Time Step 
seascape_selected = ds.isel(time=0)

# --- Prepare Colormap and Normalization ---
# Get the unique, non-NaN classes from the selected data
class_values = seascape_selected['CLASS'].values
unique_with_nan = np.unique(class_values)
unique_classes = unique_with_nan[~np.isnan(unique_with_nan)]

# Define the colormap and create a BoundaryNorm object
# This ensures each class number maps to a specific, consistent color
cmap = plt.get_cmap('tab20')
# CORRECTED: Added the '- 0.5' to center the ticks
bounds = np.append(unique_classes, unique_classes[-1] + 1)
norm = BoundaryNorm(bounds, ncolors=cmap.N)

# --- Set Up the Map ---
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
ax.set_extent([w_lon, e_lon, s_lat, n_lat], crs=ccrs.PlateCarree())

# --- Plot the Seascape Data ---
# Pass the normalization object ('norm') to ensure correct color mapping
mesh = ax.pcolormesh(
    seascape_selected['lon'], 
    seascape_selected['lat'], 
    seascape_selected['CLASS'],
    cmap=cmap,
    norm=norm, # Apply the normalization rule
    transform=ccrs.PlateCarree()
)

# --- Add Map Features ---
ax.add_feature(cfeature.LAND, zorder=1, edgecolor='black', facecolor='tan')
ax.add_feature(cfeature.COASTLINE)
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=1, color='gray', alpha=0.5, linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 12}
gl.ylabel_style = {'size': 12}

# --- Add Colorbar and Title ---
# The colorbar will automatically use the correct normalization from 'mesh'
cbar = plt.colorbar(mesh, ax=ax, ticks=unique_classes)
cbar.set_label('Seascape Class ID', fontsize=12, weight='bold')

# Get the date from the selected data
seascape_date = pd.to_datetime(seascape_selected.time.values).strftime('%Y-%m-%d')

# Set the title using the extracted date
ax.set_title(f'Seascapes for {seascape_date}', fontsize=16, weight='bold')

plt.show()

In [ ]:
# First, let's define the parameters and prepare the data structures needed for the analysis.

# --- Parameters ---
N_PIXELS = 3  # Radius in pixels to search around each station
PIX_RESOLUTION_DEG = 0.0416 # Approximate resolution of 4-km seascapes in degrees
SEARCH_RADIUS_DEG = N_PIXELS * PIX_RESOLUTION_DEG

# Prepare the seascape grid coordinates for efficient searching
# Flatten the lat/lon grid into a list of (lat, lon) points
lon_grid, lat_grid = np.meshgrid(ds['lon'].values, ds['lat'].values)
grid_points = np.vstack([lat_grid.ravel(), lon_grid.ravel()]).T

# Create a KD-Tree for fast nearest-neighbor searches. This is much faster
# than calculating the distance to every single grid point.
kdtree = KDTree(grid_points)

### USE THE NEXT CELL FOR MONTHLY SEASCAPE EXTRACTIONS

In [ ]:
# FOR MONTHLY EXTRACTIONS: This loop iterates through each month, creates the map,
#  and performs the extraction and analysis.

# List to store the final results
all_sites_summary_monthly = []

# Loop through each time step (month) in the seascape data
for i, t_row in TIMEVEC.iterrows():
    current_year = t_row['year']
    current_month = t_row['month']

    # --- Find corresponding cruise data for this month ---
    # Filter using the existing 'year' and 'month' columns, and exclude 'F' type stations
    monthly_cruise_df = ctd_df[
        (ctd_df['year'] == current_year) &
        (ctd_df['month'] == current_month) 
    ].copy()

    if monthly_cruise_df.empty:
        print(f"No cruise data for {current_year}-{current_month:02d}. Skipping.")
        continue

    print(f"Processing {len(monthly_cruise_df)} stations for {current_year}-{current_month:02d}...")

    seascape_monthly = ds.isel(time=i)
    
    # # --- A. Create the Map ---
    # fig = plt.figure(figsize=(10, 8))
    # ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
    # ax.set_extent([w_lon, e_lon, s_lat, n_lat], crs=ccrs.PlateCarree())
    
    # # Prepare BoundaryNorm for correct categorical coloring
    # class_values = seascape_monthly['CLASS'].values
    # unique_with_nan = np.unique(class_values)
    # unique_classes = unique_with_nan[~np.isnan(unique_with_nan)]
    # cmap = plt.get_cmap('tab20')
    # bounds = np.append(unique_classes, unique_classes[-1] + 1)
    # norm = BoundaryNorm(bounds, ncolors=cmap.N)

    # mesh = ax.pcolormesh(
    #     seascape_monthly['lon'], seascape_monthly['lat'], seascape_monthly['CLASS'],
    #     cmap=cmap, norm=norm, transform=ccrs.PlateCarree()
    # )
    
    # cbar = plt.colorbar(mesh, ax=ax, ticks=unique_classes, pad=0.05, aspect=25)
    # cbar.set_label('Seascape Class ID', fontsize=12)
    # ax.add_feature(cfeature.LAND, zorder=1, edgecolor='black', facecolor='tan')
    # ax.add_feature(cfeature.COASTLINE)
    # ax.gridlines(draw_labels=True, linestyle='--')
    # ax.scatter(
    #     monthly_cruise_df['lon_dec'], monthly_cruise_df['lat_dec'],
    #     s=50, c='black', marker='*', transform=ccrs.PlateCarree(),
    #     label='Cruise Stations'
    # )
    # ax.set_title(f"Seascapes and Stations: {current_year}-{current_month:02d}", fontsize=16)
    # plt.savefig(f"{current_year}-{current_month:02d}_map.png", dpi=150, bbox_inches='tight')
    # plt.close(fig)

    # --- B. Extract Surrounding Seascape Pixels ---
    station_points = monthly_cruise_df[['dec_lat', 'dec_lon']].values
    nearby_indices_list = kdtree.query_ball_point(station_points, r=SEARCH_RADIUS_DEG)
    monthly_cruise_df['nearby_grid_indices'] = nearby_indices_list
    
    # --- C. Calculate Modal Seascape Class ---
    site_summaries = []
    for idx, station in monthly_cruise_df.iterrows():
        flat_indices = station['nearby_grid_indices']
        
        # Check if any grid cells were found nearby
        if not flat_indices:
            # Case 1: No grid cells within search radius. Assign NaN and record.
            modal_class = np.nan
            mean_prob_of_modal = np.nan
        else:
            # Grid cells were found, now check if they are valid (not NaN)
            rows, cols = np.unravel_index(flat_indices, seascape_monthly['CLASS'].shape)
            nearby_classes = seascape_monthly['CLASS'].values[rows, cols]
            nearby_probs = seascape_monthly['P'].values[rows, cols]
            
            valid_mask = ~np.isnan(nearby_classes)
            valid_classes = nearby_classes[valid_mask]
            
            if valid_classes.size > 0:
                # Case 2: Valid (non-NaN) grid cells exist. Compute mode.
                modal_class = sp_mode(valid_classes, keepdims=False).mode
                valid_probs = nearby_probs[valid_mask]
                probs_of_modal_class = valid_probs[valid_classes == modal_class]
                mean_prob_of_modal = np.mean(probs_of_modal_class)
            else:
                # Case 3: All nearby grid cells were NaN. Assign NaN and record.
                modal_class = np.nan
                mean_prob_of_modal = np.nan

        # Append the result for the station, covering all cases to ensure no station is skipped
        site_summaries.append({
            'cruise_id': station['cruiseID'],
            'station': station['Station'],
            'year': station['year'],
            'month': station['month'],
            'day': station['day'],
            'time': station['time_gmt'],
            'lat_dec': station['dec_lat'],
            'lon_dec': station['dec_lon'],
            'modal_seascape_class': modal_class,
            'mean_seascape_prob': mean_prob_of_modal
        })

    all_sites_summary_monthly.extend(site_summaries)

# Convert the final list of results into a pandas DataFrame
conc_class_per_site_monthly = pd.DataFrame(all_sites_summary_monthly)

# Define the output filename
output_filename = "seascape_station_summary_monthly.tsv"

# Save the DataFrame to a .tsv file
# sep='\t' specifies a tab separator, and index=False prevents writing the row numbers
# conc_class_per_site_monthly.to_csv(output_filename, sep='\t', index=False)

print("\n--- Analysis Complete ---")
print("Final summary of modal seascape classes for 'C' type stations:")
print(conc_class_per_site_monthly.head())
print(f"\nResults saved to '{output_filename}'")

### USE THE NEXT CELL FOR 8-DAY SEASCAPE EXTRACTIONS

In [ ]:
# FOR 8-DAY EXTRACTIONS: Loop through each 8-day time window
# Convert cruise data dates to datetime objects ---
# This step is essential for comparing cruise dates to the 8-day seascape windows.
ctd_df['datetime'] = pd.to_datetime(ctd_df[['year', 'month', 'day']])

# Prepare for Storing Results ---
all_sites_summary_8day = []

# We stop at -1 because each window is defined by a start (i) and end (i+1) time
for i in range(len(ds.time) - 1):
    start_date = ds.time.values[i]
    end_date = ds.time.values[i+1]

    # --- Find corresponding cruise data within the 8-day window ---
    cruise_in_window_df = ctd_df[
        (ctd_df['datetime'] >= start_date) & 
        (ctd_df['datetime'] < end_date)
    ].copy()

    if cruise_in_window_df.empty:
        print(f"No cruise data for window starting {pd.to_datetime(start_date).strftime('%Y-%m-%d')}. Skipping.")
        continue

    print(f"Processing {len(cruise_in_window_df)} stations for window starting {pd.to_datetime(start_date).strftime('%Y-%m-%d')}...")

    # Select the seascape data for the current time step
    seascape_8day = ds.isel(time=i)
    
    # --- A. Create the Map ---
    # fig = plt.figure(figsize=(10, 8))
    # ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
    # ax.set_extent([w_lon, e_lon, s_lat, n_lat], crs=ccrs.PlateCarree())
    
    # # Use BoundaryNorm for correct categorical coloring
    # class_values = seascape_8day['CLASS'].values
    # unique_with_nan = np.unique(class_values)
    # unique_classes = unique_with_nan[~np.isnan(unique_with_nan)]
    # cmap = plt.get_cmap('tab20')
    # if unique_classes.size > 0:
    #     bounds = np.append(unique_classes, unique_classes[-1] + 1) - 0.5
    #     norm = BoundaryNorm(bounds, ncolors=cmap.N)
    #     mesh = ax.pcolormesh(seascape_8day['lon'], seascape_8day['lat'], seascape_8day['CLASS'], cmap=cmap, norm=norm, transform=ccrs.PlateCarree())
    #     cbar = plt.colorbar(mesh, ax=ax, ticks=unique_classes, pad=0.05, aspect=25)
    #     cbar.set_label('Seascape Class ID', fontsize=12)

    # ax.add_feature(cfeature.LAND, zorder=1, edgecolor='black', facecolor='tan')
    # ax.add_feature(cfeature.COASTLINE)
    # ax.gridlines(draw_labels=True, linestyle='--')
    # ax.scatter(cruise_in_window_df['lon_dec'], cruise_in_window_df['lat_dec'], s=50, c='black', marker='*', transform=ccrs.PlateCarree(), label='Cruise Stations')
    # ax.set_title(f"Seascapes and 'C' Stations: {pd.to_datetime(start_date).strftime('%Y-%m-%d')}", fontsize=16)
    # plt.savefig(f"{pd.to_datetime(start_date).strftime('%Y-%m-%d')}_C_stations_map.png", dpi=150, bbox_inches='tight')
    # plt.close(fig)

    # --- B. Extract Surrounding Seascape Pixels ---
    station_points = cruise_in_window_df[['dec_lat', 'dec_lon']].values
    nearby_indices_list = kdtree.query_ball_point(station_points, r=SEARCH_RADIUS_DEG)
    cruise_in_window_df['nearby_grid_indices'] = nearby_indices_list
    
    # --- C. Calculate Modal Seascape Class ---
    site_summaries = []
    for idx, station in cruise_in_window_df.iterrows():
        flat_indices = station['nearby_grid_indices']
        
        # Check if any grid cells were found nearby
        if not flat_indices:
            # Case 1: No grid cells within search radius. Assign NaN.
            modal_class = np.nan
            mean_prob_of_modal = np.nan
        else:
            # Grid cells were found, now check if they are valid (not NaN)
            rows, cols = np.unravel_index(flat_indices, seascape_8day['CLASS'].shape)
            nearby_classes = seascape_8day['CLASS'].values[rows, cols]
            nearby_probs = seascape_8day['P'].values[rows, cols]
            
            valid_mask = ~np.isnan(nearby_classes)
            valid_classes = nearby_classes[valid_mask]
            
            if valid_classes.size > 0:
                # Case 2: Valid (non-NaN) grid cells exist. Compute mode.
                modal_class = sp_mode(valid_classes, keepdims=False).mode
                valid_probs = nearby_probs[valid_mask]
                probs_of_modal_class = valid_probs[valid_classes == modal_class]
                mean_prob_of_modal = np.mean(probs_of_modal_class)
            else:
                # Case 3: All nearby grid cells were NaN. Assign NaN.
                modal_class = np.nan
                mean_prob_of_modal = np.nan

        # Append the result for the station, covering all cases to ensure no station is skipped
        site_summaries.append({
            'cruise_id': station['cruiseID'],
            'station': station['Station'],
            'year': station['year'], 
            'month': station['month'], 
            'day': station['day'],
            'time': station['time_gmt'],
            'lat_dec': station['dec_lat'], 
            'lon_dec': station['dec_lon'],
            'modal_seascape_class': modal_class, 
            'mean_seascape_prob': mean_prob_of_modal
        })
    all_sites_summary_8day.extend(site_summaries)

# Convert the final list of results into a pandas DataFrame
conc_class_per_site_8day = pd.DataFrame(all_sites_summary_8day)

# Define the output filename
output_filename = "seascape_station_summary_8day.tsv"

# Save the DataFrame to a .tsv file
# sep='\t' specifies a tab separator, and index=False prevents writing the row numbers
# conc_class_per_site_8day.to_csv(output_filename, sep='\t', index=False)

print("\n--- 8-Day Analysis Complete ---")
print("Final summary of modal seascape classes for 'C' type stations:")
print(conc_class_per_site_8day.head())
print(f"\nResults saved to '{output_filename}'")

## Merge monthly and 8-day seascape tables into a single file. Monthly and 8-day cells must be run prior to this merge

In [ ]:
# Start with the 8-day data as the base
merged_df = conc_class_per_site_8day.copy()

# Directly append the two columns from the monthly data
# This assumes both tables have the exact same row order
merged_df['modal_class_monthly'] = conc_class_per_site_monthly['modal_seascape_class']
merged_df['mean_prob_monthly'] = conc_class_per_site_monthly['mean_seascape_prob']

# remane the columns for 8-day seascapes
merged_df.rename(columns={'modal_seascape_class': 'modal_class_8day'}, inplace=True)  
merged_df.rename(columns={'mean_seascape_prob': 'mean_prob_8day'}, inplace=True)

# --- Save the Final Merged File ---
output_filename = "seascape_station_summary_appended.tsv"
merged_df.to_csv(output_filename, sep='\t', index=False)

print("\n--- Appending Complete ---")
print("Final appended summary:")
print(merged_df.head())
print(f"\nResults saved to '{output_filename}'")